## Export materiall from images

In [4]:
## export layers to dir named after file
import os
for file in sorted(os.listdir('images')):
    base, ext = os.path.splitext(file)
    if ext == '.svg' and base != 'test' and base != 'stack':
        print(file)
        os.system("./export_svg.sh %s" % file)
        print(os.listdir('images/m_%s' % base))

1_PS.svg
['A-3-1.png', 'A-2-1-2-1.png']
2_PSB.svg
['A-1.png', 'A-3-1.png', 'A-2-1-2-2.png']
3_PSB1.svg
['A-1.png', 'A-2-2-1.png', 'A-3-2.png']
4_PSD.svg
['A-1.png', 'A-3-2-1.png', 'A-2-1-1.png']
5_S.svg
['A-2-1-1.png']
6_SB.svg
['A-1.png', 'A-2-1-2.png']


In [15]:
def group_layers(prefix='m_', target='material', image_dir='images'):
    """
    Collect layers in separate dirs into material
    Extend name if necessary
    """
    import glob
    import shutil
    if os.path.exists(target):
        files = glob.glob(target+'/*')
        for file in files:
            os.remove(file)
    else:
        os.system('mkdir '+target)

    for d_ in sorted(os.listdir(image_dir)):
        d = image_dir + '/' + d_
        if os.path.isdir(d) and d_.startswith(prefix):
            print(d)
            for m in os.listdir(d):
                print('  %s' % m)
                if m in os.listdir(target):
                    base, ext = os.path.splitext(m)
                    num = len(glob.glob('%s/%s*.png' % (target,
                                                        base)))
                    new_m = '%s_(%s)%s' % (base, num, ext)
                else:
                    new_m = m
                shutil.move(d+'/'+m, target+'/'+new_m)
            os.rmdir(d)

In [16]:
group_layers()

images/m_1_PS
  A-3-1.png
  A-2-1-2-1.png
images/m_2_PSB
  A-1.png
  A-3-1.png
  A-2-1-2-2.png
images/m_3_PSB1
  A-1.png
  A-2-2-1.png
  A-3-2.png
images/m_4_PSD
  A-1.png
  A-3-2-1.png
  A-2-1-1.png
images/m_5_S
  A-2-1-1.png
images/m_6_SB
  A-1.png
  A-2-1-2.png


In [17]:
# Weird!
# every time use svg-objects-export.py to export a .svg, or use parse in xml to read a .svg, the objects in this image will move  to the right for a random length, but it won't affect the next read or parse, and it won't affect the appearance of exported .png files
# also, use parse, export or even open the .svg in the browser, the layers will be stacked into one layer on the very top
# seems a problem with minidom or inkscape

## Image reader

In [18]:
from xml.dom.minidom import parse
import os
import numpy as np

In [19]:
# rewrite this
# if one layer: id with svg
# if multiple layers: the first <g> with id

In [20]:
# def id_filter(id_name):
#     assert(type(id_name) is str)
#     return id_name.replace('_x3','').replace('_','')

# def get_id_list(file):
#     doc = parse(file)
    
#     e_list = [e for e in doc.getElementsByTagName('g') if e.hasAttribute('id')]
#     if e_list:
#         e0 = e_list[0]
#         for e in e_list[1:]:
#             try:
#                 assert(e in e0.parentNode.childNodes)
#             except AssertionError:
#                 print('Elements not at the same level!')
#                 print(e_list)
#                 raise AssertionError
            
#     id_list = [e.getAttribute('id') for e in e_list]
#     try:
#         assert(id_list)
#     except AssertionError:
#         id_list = [g.getAttribute('id') for g in doc.getElementsByTagName('svg') if g.hasAttribute('id')]
#         assert(id_list), 'Fatal! No id name (g or svg) available for reading!'
#         assert(len(id_list) == 1), 'Fatal! More than one element in a SVG-only case'
        
# #     try:
# #         assert(len(id_list) == len(doc.getElementsByTagName('image')))
# #     except AssertionError:
# #         print(doc.getElementsByTagName('image')) 
#     return [id_filter(id_name) for id_name in id_list]
#     # return [id_name for id_name in id_list]

In [168]:
def get_layer_names(file):
    doc = parse(file)
    svg_list = doc.getElementsByTagName('svg')
    assert(len(svg_list) == 1)
    svg = svg_list[0]
    layers = [g for g in svg.childNodes \
              if g.nodeType == 1 and \
                 g.tagName == 'g' and \
                 g.hasAttribute('id')]
    if layers:
        return [l.getAttribute('id') for l in layers]
    else:
        if svg.hasAttribute('id'):
            # if single layer case, id belongs to <svg>
            assert(len([g for g in svg.childNodes \
                          if g.nodeType == 1 and \
                             g.tagName == 'g']) == 1)
            return [svg.getAttribute('id')]
        else:
            raise ValueError('No valid id name found!')

def name2code(name):
    """
    input: A-1-2-3-4
    return: [1,2,3,4]
    """
    return [int(d) for d in name.split('-')[1:]]
    
def check_layer_names(names):
    ### styling check
    # names of layers must start with 'A-'
    for name in names:
        assert(name.startswith('A-')), "%s does not start with A-" % name
    
    # names of layers should match \d-\d
    import re
    for name in names:
        assert(re.match(r'A(-\d)+', name)), "%s does not match the pattern!" % name
    
    ### sanity check
    cat_codes = [name2code(s)[0] for s in names]
    
    # if background in, it must be the most bottom layer
    if 1 in cat_codes:
        assert(cat_codes.index(1) == 0), 'background should be the most bottom!'
        
    # if decoration in, it must be the toppest layer
    if 4 in cat_codes:
        assert(cat_codes.index(4) == len(cat_codes) - 1), 'decoration should be the most top!'
    
    if len(cat_codes) == 1:
        # if only one layer, must be surrounding layer
        assert(cat_codes[0] == 2), 'it must be the surrounding layer if there is only one layer'
    else:
        # if multiple layers, check the cat order
        # background 1 - surroundings 2 - person 3 - decoration 4    
        for code1, code2 in zip(cat_codes[:-1], cat_codes[1:]):
            assert(code2 > code1), \
               'layer %s should not be below layer %s!' % (code2, code1)

In [170]:
for fileName in sorted(glob.glob('images/*.svg')):
    # base, ext = os.path.splitext(fileName)
    # if ext == '.svg' and base != 'test' and base != 'stack':
    if 'test' in fileName or 'stack' in fileName:
        continue
    print(fileName)
    # print(get_id_list(fileName))
    names = get_layer_names(fileName)
    check_layer_names(names)
    print(names)

images/1_PS.svg
['A-2-1-2-1', 'A-3-1']
images/2_PSB.svg
['A-1', 'A-2-1-2-2', 'A-3-1']
images/3_PSB1.svg
['A-1', 'A-2-2-1', 'A-3-2-3']
images/4_PSD.svg
['A-1', 'A-2-1-1', 'A-3-2-1']
images/5_S.svg
['A-2-1-1']
images/6_SB.svg
['A-1', 'A-2-1-2-3']


In [68]:
# Caveats
# The first <image> element in a <g> is the shadow, namely the extra .png file in the dir

## Image embedding

In [164]:
def image2feature(layer_names):
    from copy import deepcopy
    
    features = []
    
    # layers
    features.append(len(layer_names))
    
    # convert to digit codes
    codes = [name2code(name) for name in layer_names]
    print(codes)
    
    # four categories, binary
    cat_feat = [0] * 4
    for code in codes:
        cat_feat[code[0] - 1] = 1
    features.append(cat_feat)
    # print(features)
    
    # 1st sub-category: 3 for surrounding and 4 for person
    # n_srd = [3, 6, 11]
    # n_prs = [4, 7]
    feat_srd = [[0,[0,0,0]],[0,[0,0,0],[0,0]]]
    feat_prs = [0,[0,0,0],[0,0,0,0]]
    for code in codes:
        
        if code[0] == 1 or code[0] == 4:
            # background or decoration, no need to encode
            assert(len(code) == 1)
        
        elif code[0] == 2:
            print('code:', code)
            # code_ = deepcopy(code)
            # print(len(code_), len(n_srd)+1)
            while len(code) < len(n_srd) + 1:
                # print('new code',code_)
                code.append(0)
            for c, n in zip(code[1:], n_srd):
                feat = [0] * n
                print('  c, n:', c,n)
                if c > 0:
                    feat[c-1] = 1
                features.append(feat)
        
        elif code[0] == 3:
            print('code:', code)
            while len(code) < len(n_prs) + 1:
                code.append(0)
            for c, n in zip(code[1:], n_prs):
                feat = [0] * n
                print('  c, n:', c,n)
                if c > 0:
                    feat[c-1] = 1
                features.append(feat)            
            
        
    print(features)
    
            
    

SyntaxError: invalid syntax (<ipython-input-164-7aa236789e9e>, line 24)

In [151]:
n_srd = {2: {2: {1, 2},
             3: {3, 1, 2}}}
n_prs = {3: {
             4: {2,1,1,1},
             3: {4,1,2}}}

In [31]:
from itertools import count, chain

def get_depth(li):
    for level in count():
        if not li:
            return level
        li = list(chain.from_iterable(e for e in li if isinstance(e, list)))

def is_pure_list(l):
#     if type(l) is not list:
#         return False
    for e in l:
        if isinstance(e, list):
            return False
    return True


def extract_1layer(feat,
                   concat=[],
                   level=0,
                   depth=4):
    level += 1
    print('  ', level, depth, concat)
    for i, e in enumerate(feat):
        if isinstance(e, list):
            if is_pure_list(e):
                if level == depth - 1:
                    concat.extend(e)
                    feat[i] = sum(e)
            else:
                feat[i], concat = extract_1layer(e, concat, level, depth)
    return feat, concat

# n_srd = [[[0],[0,0]],[[0,0,0],[0],[0,0]]]
feat_srd = [[0,[0,[0,0]]],[0,[0,0,0],[0,0]]]
# feat_prs = [0,[0,0,0],[0,0,0,0]]
feat_srd[1][2][1]=1
# print(feat_srd)

## test
# print(feat_srd)
# feat, concat = extract_1layer(feat_srd, 
#                               concat=[],
#                               level=0,
#                         depth=get_depth(feat_srd))
# print(feat, concat)
# feat, concat = extract_1layer(feat, concat=[], level=0, depth=get_depth(feat))
# print(feat, concat)

feat = feat_srd
concats = []
print(feat)
while not is_pure_list(feat):
    feat, concat = extract_1layer(feat, concat=[], level=0, depth=get_depth(feat))
    print(feat, concat)
    concats.append(concat)
concats.append(feat)
print('\nFinal concats:')
print(concats[::-1])



# def count_dfs(ns):
#     print(len(ns))
#     for n in ns:
#         if type(n) is list:
#             count_recurs(n)
# count_dfs(n_srd)
# n_srd

[[0, [0, [0, 0]]], [0, [0, 0, 0], [0, 1]]]
   1 4 []
   2 4 []
   3 4 []
   2 4 [0, 0]
[[0, [0, 0]], [0, [0, 0, 0], [0, 1]]] [0, 0]
   1 3 []
   2 3 []
   2 3 [0, 0]
[[0, 0], [0, 0, 1]] [0, 0, 0, 0, 0, 0, 1]
   1 2 []
[0, 1] [0, 0, 0, 0, 1]

Final concats:
[[0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1], [0, 0]]


In [22]:
level = 1
def test(level):
    level += 1
test(level)
print(level)

1


In [212]:
from itertools import chain
list(chain.from_iterable([[0], [[1, 2]], [2], [3, 6], [4], [5, 10]]))

[0, [1, 2], 2, 3, 6, 4, 5, 10]

In [5]:

feat_srd = [[0,[0,[0,0]]],[0,[0,0,0],[0,0]]]
print(feat_srd)
get_depth(feat_srd)

[[0, [0, [0, 0]]], [0, [0, 0, 0], [0, 0]]]


4

In [182]:
[0] * 4

[0, 0, 0, 0]

In [147]:
image2feature(['A-2-1-2-1', 'A-3-1', 'A-4'])

[[2, 1, 2, 1], [3, 1], [4]]
code: [2, 1, 2, 1]
  c, n: 1 3
  c, n: 2 6
  c, n: 1 11
code: [3, 1]
  c, n: 1 4
  c, n: 0 7
[3, [0, 1, 1, 1], [1, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]


### test text descriptions

In [21]:
['A man is lying on the sofa.',
 'A man is sitting next to a computer.',
 'A man is presenting a chart.',
 'A woman is standing next to a bucket.',
 'A deer in a sock',
 'A christmas tree with presents']

['A man is lying on the sofa.',
 'A man is sitting next to a computer.',
 'A man is presenting a chart.',
 'A woman is standing next to a bucket.',
 'A deer in a sock',
 'A christmas tree with presents']

## Renderer

In [42]:
def str2num_size(size):
    assert type(size) is str
    return int(float(size.strip('pt')))
    
def get_size(file):
    """
    Get the width and height of an image
    
    .svg:
        use xml's parse
        By check the 'viewBox' attribute in the 'svg' tag
    
    .png:
        use PIL
    """
    
    basename, ext = os.path.splitext(file)
    if ext == '.svg':   
        from xml.dom.minidom import parse
        doc = parse(file)
        # search svg element
        image_list = doc.getElementsByTagName('svg')
        assert(image_list), 'no svg element found!'
        assert(len(image_list) == 1), file
        img = image_list[0]
        assert(img.hasAttribute('viewBox'))
        width, height = img.getAttribute('viewBox').split()[2:]
        width, height = (str2num_size(width), str2num_size(height))
        # try search image element, then g element
#     try:
#         assert image_list
#     except AssertionError:
#         image_list = doc.getElementsByTagName('svg')
#         assert image_list
#         assert image_list[0].hasAttribute('width')
#     assert(len(image_list) == 1), svg_file
#     ## if multiple images, size is the maximum size in either direction        
    elif ext == '.png':
        from PIL import Image
        width, height = Image.open(file).size

    return (width, height)

In [43]:
def stack_svgs(file_list, opt_file=None, canvas_size=None):
 
    import cairosvg
    from svgutils.compose import Figure, Image #,SVG
    
    if not opt_file:
        opt_file = 'stack.svg'
        
    if canvas_size:
        canvas_w, canvas_h = canvas_size
    else:
        canvas_w, canvas_h = 0, 0
        for file in file_list:
            width, height = get_size(file)
            if width > canvas_w: canvas_w = width
            if height > canvas_h: canvas_h = height
    print('Canvas size:', (canvas_w, canvas_h))
    
    # if svg, convert to png first
    file_list_png = []
    for file in file_list:
        basename, ext = os.path.splitext(file)
        if ext == '.svg':
            png_file = basename + '.png'
            cairosvg.svg2png(url=file,
                             write_to=png_file)
            file_list_png.append(png_file)
        elif ext == '.png':
            file_list_png.append(file)
        else:
            raise ValueError('File type not availale!')
    
    image_list = []
    # 
    for file in file_list_png:
        print('File:', file)
        width, height = get_size(file)
        img = Image(width, height, file)
        img.move(int((canvas_w-width)/2),
                 int((canvas_h-height)/2))
        image_list.append(img)
    
        
    Figure(canvas_w, canvas_h, *image_list).save(opt_file)

In [46]:
stack_svgs(['svg-objects-export/A-1.png', 
            'svg-objects-export/A-2-1-2-1.png', 
            'svg-objects-export/A-3-1.png'])

Canvas size: (1114, 786)
File: svg-objects-export/A-1.png
File: svg-objects-export/A-2-1-2-1.png
File: svg-objects-export/A-3-1.png


In [45]:
stack_svgs(['material/背景4.svg', 
            'material/123.svg', 
            'material/生活方式3.svg'])

Canvas size: (1145, 816)
File: material/背景4.png
File: material/123.png
File: material/生活方式3.png


/Users/dongjustin/miniconda2/envs/text2scene/lib/python3.7/site-packages/cairosvg/parser.py:416: UserWarning: No tag with id=未命名的渐变_12-2 found.
  warnings.warn('No tag with id={} found.'.format(element_id))
/Users/dongjustin/miniconda2/envs/text2scene/lib/python3.7/site-packages/cairosvg/parser.py:416: UserWarning: No tag with id=未命名的渐变_12-3 found.
  warnings.warn('No tag with id={} found.'.format(element_id))
/Users/dongjustin/miniconda2/envs/text2scene/lib/python3.7/site-packages/cairosvg/parser.py:416: UserWarning: No tag with id=未命名的渐变_12-4 found.
  warnings.warn('No tag with id={} found.'.format(element_id))


In [13]:
## write a tool to stack two svgs:
### put <g> altogether, then for each elememt, add attribute recursively

In [354]:
## ValueError: material/124.svg
## issue of cairosvg converter

## Test

In [132]:
doc = parse('SB.svg')
# ele = [c for c in doc.childNodes if c.nodeType == 1]
# assert(len(ele) == 1)
# [n.getAttribute('id') for n in ele[0].childNodes if n.nodeType==1 and n.tagName=='g']
# [g.getAttribute('id') for g in doc.getElementsByTagName('g') if g.hasAttribute('id')]
# id_list = [g.getAttribute('id') for g in doc.getElementsByTagName('g') if g.hasAttribute('id')]
# assert(id_list)

# get the tagname: element.tagName

In [76]:
# the order may be reversed, that's weird
# at least. background must be at the bottom
# but what if there are only person and surronding

# it's reversed
# because the first object is at the bottom

In [74]:
doc.getElementsByTagName('image')

[<DOM Element: image at 0x112681cc0>]

In [56]:
[g.getAttribute('id') for g in doc.getElementsByTagName('*') if g.hasAttribute('id')]

['_x31__x5F_1_x5F_1']